# Import packages

In [ ]:
# load data
from submodules.load_data import load_data

# data manipulation
import numpy as np
import pandas as pd

# data splitting
from sklearn.model_selection import train_test_split

# data preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# model
from xgboost import XGBClassifier

# hyperparameter tuning
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV

# k-fold cross validation
from sklearn.model_selection import cross_validate

# saving models
import joblib

# performance
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt


# Load the data

Load semi-colon separated data from disk

In [ ]:
data = load_data()

# Create a Test Dataset

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(data.drop("isSepsis", axis=1),
    data["isSepsis"], test_size=0.15,
    random_state=42, stratify=data["isSepsis"])

# Drop the non-biological attributes from the training data

In [ ]:
data_num = X_train.drop(["Age",
                         "Unit1",
                         "Unit2",
                         "HospAdmTime",
                         "ICULOS",
                         "Gender"
                         ], axis=1)

# Transform the numeric training data

In [ ]:
num_imputer = SimpleImputer(strategy="median")
num_imputer.fit(data_num)
N = num_imputer.transform(data_num)

# Transformation pipeline

In [ ]:
num_pipeline = Pipeline([
                        ('imputer', SimpleImputer(strategy='median')),
                        ('std_scaler', StandardScaler()),
                        ])

num_prepared = num_pipeline.fit_transform(data_num)

# Full data pipeline

In [ ]:
num_attribs = list(data_num)
# construct the transformer
full_pipeline = ColumnTransformer([
    # transform number columns with num_pipeline defined earlier
    ("num", num_pipeline, num_attribs)
])

# only run the pipeline on the training as the test data will be applied during the evaluation stage with the final model
X_train = full_pipeline.fit_transform(X_train)

# Train a Random Forest Classifier

In [ ]:
model = XGBClassifier()
cv_model = cross_validate(model,
                       X_train,
                       y_train,
                       n_jobs=-1,
                       cv=3,
                       scoring="f1",
                       return_train_score=True)
cv_model

In [ ]:
# serialize the model
joblib.dump(model, "models/tune/rfc_model.pkl")

In [ ]:
# load the model from disk
rfc_model = joblib.load("models/tune/rfc_model.pkl")

# Fine tune the Random Forest Classifier with Randomize Search Cross Validation

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html?highlight=randomizedsearchcv#sklearn.model_selection.RandomizedSearchCV
https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [ ]:
print("[INFO] setting hyperparameters...")
# number of trees in the forest
n_estimators = [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
# max number of features considered for splitting a node
max_features = ["auto","sqrt","log2"]
# max number of levels in each decision tree
max_depth = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]
# min number of data points placed in a node before the node is split
min_samples_split = [2, 5, 10]
# min number of data points allowed in a leaf node
min_samples_leaf = [1, 2, 4]
# method for sampling data points (with or without replacement)
bootstrap = [True, False]
grid = dict(n_estimators=n_estimators, max_features=max_features, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, bootstrap=bootstrap)

In [ ]:
# initialize a cross-validation fold and perform a randomized-search
# to tune the hyperparameters
print("[INFO] grid searching over the hyperparameters...")
cvFold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
randomSearch = RandomizedSearchCV(estimator=model, n_jobs=-1,
	cv=cvFold, param_distributions=grid,
	scoring="f1", return_train_score=True)
searchResults = randomSearch.fit(X_train, y_train)

In [ ]:
# print the best score
searchResults.best_score_

In [ ]:
# print the best combination of parameters
searchResults.best_params_

In [ ]:
# print the best estimator directly
searchResults.best_estimator_

In [ ]:
# fit the data with the best model parameters
rfc_model = RandomForestClassifier(**searchResults.best_params_)
rfc_model.fit(X_train, y_train)

In [ ]:
# confusion matrix
fig, ax = plt.subplots(figsize=(6, 6))
plot_confusion_matrix(rfc_model, X_test, y_test, cmap="Blues", ax=ax)
plt.savefig("reports/confusionMatrix.png", dpi=400)

In [ ]:
# display the importance scores next to the corresponding attributes
# from this you can drop less useful features
feature_importances = searchResults.best_estimator_.feature_importances_
sorted(zip(feature_importances, num_attribs), reverse=True)

Evaluate the Random Forest Classifier

In [ ]:
# transform, DON'T fit the final data
X_test = full_pipeline.transform(X_test)

In [ ]:
model_rfc = searchResults.best_estimator_

rfc_predictions = model_rfc.predict(X_test)

In [ ]:
# view the percent of the predictions that were correct
accuracy_score(y_test, rfc_predictions)

In [ ]:
# we can successfully identify 6 out of 10 patients that will develop sepsis in the next 6 days
print(recall_score(y_test, rfc_predictions))
print(f1_score(y_test, rfc_predictions))

In [ ]:
# save the model
joblib.dump(model_rfc, "models/final/model_rfc.pkl")
# reference to load the model
#final_rfc_loaded = joblib.load("models/final/model_rfc.pkl")

